In [ ]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, recall_score, make_scorer, precision_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV